# Run batches of FastScape simulations

Via xarray-simlab, we leverage the ``dask`` library (https://docs.dask.org/en/latest/) and its integration with ``xarray`` to easily run, analyse and visualize batches of model runs, e.g., in the cases of sentitivity analyses or inversions.

Let's import some packages first:

In [ ]:
import numpy as np
import xarray as xr
import xsimlab as xs
import fastscape

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

You won't need to run the cell below when support for running batches of simulations will be added in ``xarray-simlab``.

## Import the model and the model base setup

Let's import the basic (standard) FastScape model

In [ ]:
from fastscape.models import basic_model

The setup create in the ``run_basic_model`` is reused here as a base setup

In [ ]:
in_ds = xr.load_dataset('basic_input.nc')

## Example 1: run models for different values of $K$ (stream power law)

We just need to set the corresponding variable with different values and a dimension label for the batch, and then pass this dimension label to ``.xsimlab.run``. To run the models in parallel we set ``parallel=True`` and we choose here Dask's ``processes`` scheduler. With this scheduler, we also need to set ``store`` with a compatible storage alternative (i.e., not the default, in-memory store).

In [ ]:
in_vars = {'spl__k_coef': ('spl__k_coef', np.linspace(1e-5, 1e-4, 5))}

In [ ]:
from dask.diagnostics import ProgressBar

with basic_model, ProgressBar():
    out_ds = (
        in_ds
        .xsimlab.update_vars(input_vars=in_vars)
        .xsimlab.run(batch_dim='spl__k_coef', parallel=True, scheduler="processes", store="batch.zarr")
    )
    
out_ds

Plotting using ``hvplot`` is just as easy as with single model runs. We can easily explore the parameter space.

In [ ]:
import hvplot.xarray
import matplotlib.pyplot as plt

out_ds.topography__elevation.hvplot.image(
    x='x', y='y', cmap='viridis', groupby=['spl__k_coef', 'out'])

Extracting statistics along the batch dimension is very easy (and it's executed in parallel):

In [ ]:
avg = out_ds.topography__elevation.mean(dim='spl__k_coef')


Selecting and plotting cross-sections is easy too:

In [ ]:
avg.isel(out=-1).sel(x=10000).plot();

Xarray has powerful plotting capabilities implemented on top of matplotlib:

In [ ]:
(out_ds
 .topography__elevation.isel(out=-1)            # select elevation at the end of the simulation
 .mean(dim='x')                                 # average over the 'x' axis
 .plot(col='spl__k_coef', col_wrap=4)           # plot for all simulations in the batch (K coef values)
);      